In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# Standard Score
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [2]:
raw_data="""Rank Name Age Height
(scaled)
Weight
(lb)
Reach
(cm)
1 Alexander Volkanovski 33 1.00 144.5 182
2 Islam Makhachev 31 4.60 154.5 179
3 Leon Edwards 31 6.40 169.5 188
4 Kamaru Usman 35 6.40 170 193
5 Francis Ngannou 35 10.00 257 211
6 Israel Adesanya 33 10.00 185 203
7 Charles Oliveira 33 4.60 175 188
8 Alex Pereira 35 9.64 184.6 200
9 Aljamain Sterling 33 1.72 153 180
10 Jiří Procházka 30 8.92 205 203"""

In [3]:
# data = paste()
data = raw_data.split("\n")
columns = data[0].split()
for item in list(data[1:]):
    if item.startswith("("):
        columns[-1] += f" {item}"
        data.remove(item)
    elif item[0].isdigit():
        break
    else:
        columns.append(item)
        data.remove(item)
data = data[1:]
data = [x.split() for x in data]
data = [x[:1] + [" ".join(x[1:3])] + x[3:] for x in data]
data = pd.DataFrame(data, columns=columns)
for type_,col in zip([int,str,int,float,float,int],data.columns):
    data[col] = data[col].astype(type_)

display(data.rename(columns={col:f"{col} ({data[col].dtype})" for col in data.columns}))

,Rank (int32),Name (object),Age (int32),Height (scaled) (float64),Weight (lb) (float64),Reach (cm) (int32)
0,1,Alexander Volkanovski,33,1.00,144.5,182
1,2,Islam Makhachev,31,4.60,154.5,179
2,3,Leon Edwards,31,6.40,169.5,188
3,4,Kamaru Usman,35,6.40,170.0,193
4,5,Francis Ngannou,35,10.00,257.0,211
5,6,Israel Adesanya,33,10.00,185.0,203
6,7,Charles Oliveira,33,4.60,175.0,188
7,8,Alex Pereira,35,9.64,184.6,200
8,9,Aljamain Sterling,33,1.72,153.0,180
9,10,Jiří Procházka,30,8.92,205.0,203


In [4]:
# (a) Equal width binning
lower_bound_open = 125
upper_bound_closed = 265
n_bins = 4
interval_width = (upper_bound_closed - lower_bound_open) / n_bins
lower_bounds = [lower_bound_open + interval_width * i for i in range(n_bins)]
upper_bounds = [lower_bound_open + interval_width * (i+1) for i in range(n_bins)]
col_ = data.columns[4]

bins = []
def get_df_bin(df, col, lower_bound, upper_bound):
    return df[(df[col] > lower_bound) & (df[col] <= upper_bound)]

for i in range(n_bins):
    print(f"({lower_bounds[i]},{upper_bounds[i]}]")
    display(get_df_bin(data, col_, lower_bounds[i], upper_bounds[i]))

(125.0,160.0]


,Rank,Name,Age,Height (scaled),Weight (lb),Reach (cm)
0,1,Alexander Volkanovski,33,1.00,144.5,182
1,2,Islam Makhachev,31,4.60,154.5,179
8,9,Aljamain Sterling,33,1.72,153.0,180


(160.0,195.0]


,Rank,Name,Age,Height (scaled),Weight (lb),Reach (cm)
2,3,Leon Edwards,31,6.40,169.5,188
3,4,Kamaru Usman,35,6.40,170.0,193
5,6,Israel Adesanya,33,10.00,185.0,203
6,7,Charles Oliveira,33,4.60,175.0,188
7,8,Alex Pereira,35,9.64,184.6,200


(195.0,230.0]


,Rank,Name,Age,Height (scaled),Weight (lb),Reach (cm)
9,10,Jiří Procházka,30,8.92,205.0,203


(230.0,265.0]


,Rank,Name,Age,Height (scaled),Weight (lb),Reach (cm)
4,5,Francis Ngannou,35,10.0,257.0,211


In [5]:
# (b) Min-max reverse scaling
l,u = 168,193
col = data.columns[3]
scaler = MinMaxScaler(feature_range=(l,u)).fit(data[[col]])

idxs = [4,6,8]
idxs = [x-1 for x in idxs]

df_b = data.copy()
df_b["Height (original)"] = scaler.transform(df_b[[col]])
# Highlight rows of df_b in idxs
# float_cols = [x for x in df_b.columns if df_b[x].dtype == float]
df_b_style = df_b.style.apply(lambda x: ["background: gray" if x.name in idxs else "" for i in x], axis=1)
df_b_style = df_b_style.format({col: "{:.2f}" for col in [x for x in df_b.columns if df_b[x].dtype == float]})
display(df_b_style)

,Rank,Name,Age,Height (scaled),Weight (lb),Reach (cm),Height (original)
0,1,Alexander Volkanovski,33,1.00,144.50,182,168.00
1,2,Islam Makhachev,31,4.60,154.50,179,178.00
2,3,Leon Edwards,31,6.40,169.50,188,183.00
3,4,Kamaru Usman,35,6.40,170.00,193,183.00
4,5,Francis Ngannou,35,10.00,257.00,211,193.00
5,6,Israel Adesanya,33,10.00,185.00,203,193.00
6,7,Charles Oliveira,33,4.60,175.00,188,178.00
7,8,Alex Pereira,35,9.64,184.60,200,192.00
8,9,Aljamain Sterling,33,1.72,153.00,180,170.00
9,10,Jiří Procházka,30,8.92,205.00,203,190.00


In [6]:
# (c) Standard Score
col = data.columns[5]
df_c = data.copy()
mean = df_c[col].mean()
std = df_c[col].std()
std_given = 10.53
scaler = StandardScaler().fit(df_c[[col]])
df_c["Reach (standard score)"] = scaler.transform(df_c[[col]])
my_scaler = lambda x: (x - mean) / std_given
df_c["Reach (standard score with given std)"] = df_c[col].apply(my_scaler)

idxs = list(map(lambda x: x-1, [3,5]))
df_c_style = df_c.style.apply(lambda x: ["background: gray" if x.name in idxs else "" for i in x], axis=1)
df_c_style = df_c_style.format({col: "{:.2f}" for col in [x for x in df_c.columns if df_c[x].dtype == float]})
print(f"Mean: {mean:.2f}, Std: {std:.2f}, Std (given): {std_given:.2f}")
display(df_c_style)

Mean: 192.70, Std: 11.10, Std (given): 10.53


,Rank,Name,Age,Height (scaled),Weight (lb),Reach (cm),Reach (standard score),Reach (standard score with given std)
0,1,Alexander Volkanovski,33,1.00,144.50,182,-1.02,-1.02
1,2,Islam Makhachev,31,4.60,154.50,179,-1.30,-1.30
2,3,Leon Edwards,31,6.40,169.50,188,-0.45,-0.45
3,4,Kamaru Usman,35,6.40,170.00,193,0.03,0.03
4,5,Francis Ngannou,35,10.00,257.00,211,1.74,1.74
5,6,Israel Adesanya,33,10.00,185.00,203,0.98,0.98
6,7,Charles Oliveira,33,4.60,175.00,188,-0.45,-0.45
7,8,Alex Pereira,35,9.64,184.60,200,0.69,0.69
8,9,Aljamain Sterling,33,1.72,153.00,180,-1.21,-1.21
9,10,Jiří Procházka,30,8.92,205.00,203,0.98,0.98
